This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'your API key here'

#Note: remove API key before turning in assignment

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests


In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned

url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?api_key='+API_KEY
r = requests.get(url)
json_data = r.json()

type(json_data)
#json_data is a dictionary

json_data.keys()
#there is only one key: 'dataset_data'

json_data['dataset_data'].keys()
#there are multiple keys, but the one we're interested in is the 'data'

json_data['dataset_data']['data'][0]
#returns the data for today's date

['2020-08-14',
 92.9,
 93.25,
 91.35,
 91.85,
 None,
 78700.0,
 7257628.1,
 None,
 None,
 None]

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure

#json_data
#commented out "json_data" to make notebook shorter and easier to grade

#  'column_names': ['Date',
#   'Open',
#   'High',
#   'Low',
#   'Close',
#   'Change',
#   'Traded Volume',
#   'Turnover',
#   'Last Price of the Day',
#   'Daily Traded Units',
#   'Daily Turnover'],

#  'start_date': '2000-06-07',
#  'end_date': '2020-08-14',
#  'frequency': 'daily',
#  'data':[list of lists. Each sub-list is a date with that day's data.]

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [5]:
#1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 
#(keep in mind that the date format is YYYY-MM-DD).

url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01&end_date=2017-12-31&api_key='+API_KEY
r = requests.get(url)
json_data = r.json()
#json_data


In [6]:
#2. Convert the returned JSON object into a Python dictionary.

type(json_data)

#the JSON object is already a dictionary

dict

In [7]:
#3. Calculate what the highest and lowest opening prices were for the stock in this period.

#We make a dictionary, where key = date, and value = opening price
dict_open = {}

for i in range(len(json_data['dataset_data']['data'])):
    key = json_data['dataset_data']['data'][i][0]
    value = json_data['dataset_data']['data'][i][1]
    
    if type(value) == float:
        dict_open[key] = value
    else:
        pass

max_value = max(dict_open.values())
min_value = min(dict_open.values())

print("The highest opening price was $" + str(max_value) + " and the lowest was $" + str(min_value))


The highest opening price was $53.11 and the lowest was $34.0


In [8]:
#4. What was the largest change in any one day (based on High and Low price)?

#Note: high and low price are index 2 and 3 in: json_data['dataset_data']['data'][(date)][(index)] 
#We can do something similar to the previous exercise, where we make a dictionary
#where key = date and value = the difference in high and low price.

dict_change = {}

for i in range(len(json_data['dataset_data']['data'])):
    key = json_data['dataset_data']['data'][i][0]
    high = json_data['dataset_data']['data'][i][2]
    low = json_data['dataset_data']['data'][i][3]
    value = high - low
    
    if type(value) == float:
        dict_change[key] = value
    else:
        pass

max_value = max(dict_change.values())
print('The largest change on any one day, based on High/Low price, was $' + str(round(max_value, 2)))


The largest change on any one day, based on High/Low price, was $2.81


In [9]:
#5. What was the largest change between any two days (based on Closing Price)?

#Same as above... this time, closing price is the 4th index.
#Since we're finding the difference between 2 days, let's skip the last day by adding '-1' to our range.
#Skipping the last day means each value is the difference between today and tomorrow's closing price.

dict_change_close = {}

for i in range(len(json_data['dataset_data']['data'])-1):
    key = json_data['dataset_data']['data'][i][0]
    close_tomorrow = json_data['dataset_data']['data'][i+1][4]
    close_today = json_data['dataset_data']['data'][i][4]
    value = close_tomorrow - close_today
    
    if type(value) == float:
        dict_change_close[key] = value
    else:
        pass

max_value = max(dict_change_close.values())
min_value = min(dict_change_close.values())

print(max_value)
print(min_value)
print('The largest change between any two days, based on Closing Price, was $' + str(round(max_value, 2)))


2.559999999999995
-1.7199999999999989
The largest change between any two days, based on Closing Price, was $2.56


In [10]:
#6. What was the average daily trading volume during this year?

#Note: the Traded Volume is index 6 in: json_data['dataset_data']['data'][(date)][(index)] 
#let's make a list of all the values and then find the average.

traded_vol = []
for i in range(len(json_data['dataset_data']['data'])):
    value = json_data['dataset_data']['data'][i][6] 
    if type(value) == float:
        traded_vol.append(value)
    else:
        pass

avg = sum(traded_vol) / len(traded_vol)

print('The average daily trading volume during 2017 was $' + str(round(avg, 2)))


The average daily trading volume during 2017 was $89124.34


In [11]:
#7. (Optional) What was the median trading volume during this year. 
#(Note: you may need to implement your own function for calculating the median.)

def median(lst):
    lst_sorted = sorted(lst)
    lst_length = len(lst)
    index = (lst_length - 1) // 2

    if (lst_length % 2):
        return lst_sorted[index]
    else:
        return (lst_sorted[index] + lst_sorted[index + 1])/2.0

med = median(traded_vol)
print('The median daily trading volume during 2017 was $' + str(round(med, 2)))


The median daily trading volume during 2017 was $76286.0
